In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
data = pd.read_parquet(r"fridge_data_cleaned.parquet")
#data = pd.read_csv(r"D:\Python\Fridge_e_cons\fridge_data_cleaned.csv")

print(data.head())

In [ ]:
## Changes made for appliances names which were not correctly extracted because of upper/lower case and spaces

""" 
data['name'] = data['name'].str.strip()
data['name'] = data['name'].str.replace(r'\s+', ' ', regex=True)  # Normalize spaces


# Step 2: Extract "fryser" or "køleskab" with their numbers
# data['extracted_appliance'] = data['name'].str.extract(r'(fryser\s*\d+|køleskab\s*\d+)$')

data['extracted_appliance'] = data['name'].str.extract(r'(?i)(fryser\s*\d+|køleskab\s*\d+)\s*$', expand=False)


# Step 3: Update the 'appliances' column where it is NaN
data['appliance'] = data['appliance'].fillna(data['extracted_appliance'])

data['appliance'] = (data['appliance'].str.lower().str.replace(r'(fryser|køleskab)\s*(\d+)', r'\1 \2', regex=True).str.strip())


# Remove the extracted appliance from the 'name' column
#data['name'] = data['name'].str.replace(r'(fryser\s*\d+|køleskab\s*\d+)$', '', regex=True).str.strip()

data['name'] = data['name'].str.replace(r'(?i)(fryser\s*\d+|køleskab\s*\d+)\s*$', '', regex=True).str.strip()


# Step 5: Drop the temporary column (optional)
data = data.drop(columns=['extracted_appliance'])

print(data.head(10))

"""

In [ ]:
# Clean the 'name' column

data['name'] = data['name'].str.strip()
data['name'] = data['name'].str.replace(r'\s+', ' ', regex=True)  # Normalize spaces


# Step 2: Extract "fryser" or "køleskab" with their numbers
# data['extracted_appliance'] = data['name'].str.extract(r'(fryser\s*\d+|køleskab\s*\d+)$')

data['extracted_appliance'] = data['name'].str.extract(r'(?i)(fryser\s*\d+|køleskab\s*\d+)\s*$', expand=False)

# Step 3: Update the 'appliances' column where it is NaN
data['appliance'] = data['appliance'].fillna(data['extracted_appliance'])

# Step 4: Remove the extracted appliance from the 'name' column
#data['name'] = data['name'].str.replace(r'(fryser\s*\d+|køleskab\s*\d+)$', '', regex=True).str.strip()

data['name'] = data['name'].str.replace(r'(?i)(fryser\s*\d+|køleskab\s*\d+)\s*$', '', regex=True).str.strip()


# Step 5: Drop the temporary column (optional)
data = data.drop(columns=['extracted_appliance'])

print(data.head(10))

In [ ]:
## group the data by places

# List of places
places = [
    'Bofællesskabet Mårslet', 'Bofællesskabet Vintervej', 'Børnehaven Hygge og Leg', 
    'Børnehuset Bakken', 'Boernesymfonien', 'CSB Brabrand', 'CSB Egebækken', 
    'Graven', 'Harlev', 'Jespers Torvekøkken', 'Kantinen Grøndalsvej 2', 
    'Plejehjemmet Skovvang', 'Plejehjemmet Trøjborg', 'Vikærgården'
]

# Create a new column 'place' by checking if any place name is in the 'name' column
data['place'] = data['name'].apply(lambda x: next((place for place in places if place in x), 'Unknown'))

# Step 2: Group by 'place'
grouped = data.groupby('place')

print(type(grouped))
print(grouped.head())
print(grouped.size())

In [ ]:
# Filter data for a specific place (e.g., "Graven")

selected_place = "Graven"
filtered_df = data[data['place'] == selected_place]


# This assumes the kind is the first part of the appliance name
filtered_df['appliance_kind'] = filtered_df['appliance'].str.extract(r'([\wæøåÆØÅ]+)')


In [ ]:
# Create separate plots for each kind of appliance
for appliance_kind, group in filtered_df.groupby('appliance_kind'):
    fig = go.Figure()

    # Add a trace for each individual appliance (e.g., fryser1, fryser2)
    for appliance, appliance_group in group.groupby('appliance'):
        fig.add_trace(go.Scatter(
            x=appliance_group['time_index'],
            y=appliance_group['temperature'],
            mode='lines+markers',
            name=appliance  # Use appliance name as the trace name
        ))

    # Customize the plot
    fig.update_layout(
        title=f"Temperature vs Time for {selected_place} ({appliance_kind})",
        xaxis_title="Time",
        yaxis_title="Temperature (°C)",
        legend_title="Appliance Unit",
        template="plotly_white"
    )

    # Show the plot for this appliance kind
    fig.show()